# Add Opportunity Zone and Rising Tide Scores

# University of Michigan
## School of Information
**Masters of Applied Data Science**<br>
**Milestone 1**<br>
<br>
Affordable Housing Project
<br><br>
**Submitted by**<br>
>Alan Fehsenfeld<br>Koon Leong Ho<br>George Thio

In [1]:
import geopandas
import pandas as pd
from config import basedir

base = basedir()

## Read Census Tract Shape Files and Tax File and Join Them

In [72]:
data_dir = base + r"/data/raw/shape_files"
fp_input_file = data_dir + r"/tl_2019_26_tract.shp"

# michigan_tracts=geopandas.read_file('documents/UofM/SIADS 591/tl_2019_26_tract.shp')

michigan_tracts=geopandas.read_file(fp_input_file)
michigan_tracts=michigan_tracts.to_crs('EPSG:2253')

In [73]:
data_dir = base + r"/data/municipalities"
fp_input_file = data_dir + r"/All Municipalities Geometry Only.shp"
Municipalities=geopandas.read_file(fp_input_file)

# Municipalities=geopandas.read_file('documents/UofM/SIADS 591/All Municipalities Geometry Only.shp')

In [100]:
Municipalities_file_with_tract=geopandas.sjoin(Municipalities,michigan_tracts,how='inner',op='intersects')
Municipalities_file_with_tract.loc[:,['COUNTYFP','TRACTCE']]=Municipalities_file_with_tract.loc[:,['COUNTYFP','TRACTCE']].astype(int)

In [101]:
Municipalities_file_with_tract['GEOID']=Municipalities_file_with_tract['GEOID'].astype(int)

## Read Opportunity Zone File and Merge with Tax File

In [191]:
data_dir = base + r"/data/raw"
fp_input_file = data_dir + r"/Designated QOZs.12.14.18.csv"

# opportunity_zones=pd.read_csv('documents/UofM/SIADS 591/Designated QOZs.12.14.18.csv')

opportunity_zones=pd.read_csv(fp_input_file)
Municipalities_file_with_tract2=Municipalities_file_with_tract.merge(opportunity_zones,how='left',left_on='GEOID',right_on='Census Tract Number')
Municipalities_file_with_tract2=Municipalities_file_with_tract2.loc[:,['ID','State']]
Municipalities_file_with_tract2=Municipalities_file_with_tract2.rename(columns={'State':'oz_ind'})

In [196]:
Municipalities_file_with_tract2['oz_ind']=Municipalities_file_with_tract2['oz_ind'].where(Municipalities_file_with_tract2['oz_ind'].isna(),3)
Municipalities_file_with_tract2['oz_ind']=Municipalities_file_with_tract2['oz_ind'].where(Municipalities_file_with_tract2['oz_ind'].notna(),0)
Municipalities_file_with_tract2=Municipalities_file_with_tract2.groupby('ID').agg('max').reset_index()

In [199]:
data_dir = base + r"/data"
fp_input_file = data_dir + r"/Opportunity Zones.csv"
Municipalities_file_with_tract2.to_csv('documents/UofM/SIADS 591/Opportunity Zones.csv',index=False)

# Municipalities_file_with_tract2.to_csv('documents/UofM/SIADS 591/Opportunity Zones.csv',index=False)